In [ ]:
from fastapi import FastAPI, File, UploadFile, WebSocket, HTTPException, WebSocketDisconnect
from fastapi.responses import HTMLResponse, JSONResponse, FileResponse
from fastapi.middleware.cors import CORSMiddleware
from nbconvert.preprocessors import ExecutePreprocessor
import nbformat as nb
import asyncio
import uvicorn
import os
from typing import List
import shutil
import aiofiles

app = FastAPI()

UPLOAD_DIR = "uploads"
ausgabebild_filepath = "./Ausgabebild.png"

async def execute_notebook(notebook_path, output_image_path, websocket):
    try:
        # Laden des Notebooks
        with open(notebook_path, 'r', encoding='utf-8') as f:
            nb_read = nb.read(f, as_version=4)

        # Asynchrone Ausführung des Notebooks
        ep = ExecutePreprocessor(timeout=600, kernel_name='python3')
        ep.preprocess(nb_read, {'metadata': {'path': './'}})  # Pfad ggf. anpassen

    except Exception as e:
        print(f"Fehler bei der Ausführung des Notebooks: {e}")


async def send_output_image(output_image_path, websocket):
    try:
        with open(output_image_path, 'rb') as f:
            image_bytes = f.read()
        await websocket.send_bytes(image_bytes)
        print("Ausgabebild erfolgreich gesendet.")
    except FileNotFoundError:
        print(f"Das Ausgabebild unter {output_image_path} wurde nicht gefunden.")
    except Exception as e:
        print(f"Ein Fehler ist aufgetreten beim Senden des Ausgabebilds: {str(e)}")

@app.websocket("/ws/connection-test")
async def websocket_conncetion_test(websocket: WebSocket):
    await websocket.accept()
    message = await websocket.receive_text() 
    try:
        if message == "connection test":
            await websocket.send_text("Verbindung erfolgreich.")
        else:
            await websocket.close()
    except Exception as e:
        print(f"Ein Fehler ist beim Verbindungsversuch aufgetreten: {str(e)}")


@app.websocket("/ws/endpoint")
async def unified_websocket_endpoint(websocket: WebSocket):
    await websocket.accept()
    try:
        while True:
            # Empfangen von Nachrichten als Text
            message = await websocket.receive_text()
            message_data = message.split(':')  # Nehmen wir an, Nachrichten sind im Format "Befehl:Inhalt" formatiert

            command = message_data[0]
            if command == "run notebook":
                notebook_path = "StyleTransferForApp.ipynb"
                output_image_path = "Ausgabebild.png"

                # Starten der Notebook-Ausführung als asynchrone Task
                notebook_execution_task = asyncio.create_task(execute_notebook(notebook_path, output_image_path, websocket))
                await notebook_execution_task
                await websocket.send_text("Notebook erfolgreich ausgeführt.")
                # Aufrufen der Funktion zum Senden des Ausgabebildes direkt nach der Erfolgsmeldung
                await send_output_image(output_image_path, websocket)

            elif command == "send style image" or command == "send content image":
                # Der Rest der Nachricht wird als Binärdaten für das Bild behandelt
                image_data = await websocket.receive_bytes()  # Umwandlung zurück in Binärdaten

                # Bestimmen des Dateinamens basierend auf dem Befehl
                image_filename = "styleimage.jpg" if command == "send style image" else "contentimage.jpg"
                image_path = os.path.join(UPLOAD_DIR, image_filename)

                async with aiofiles.open(image_path, "wb") as f:
                    await f.write(image_data)
                if command == "send style image":
                    await websocket.send_text(f"Stylebild erfolgreich hochgeladen")
                elif command == "send content image":
                    await websocket.send_text(f"Contentbild erfolgreich hochgeladen")

            else:
                await websocket.send_text("Unbekannte Anfrage.")

    except WebSocketDisconnect:
        print("Client disconnected")
        await websocket.close()

        
if __name__ == "__main__":
    config = uvicorn.Config(app, host="0.0.0.0", port=8810, log_level="info")
    server = uvicorn.Server(config)
    await server.serve()
